In [1]:
import pandas as pd
import numpy as np
df  = pd.read_csv(r'I:/Hawkes_Model_MM/hawkes_analysis_data.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
dates = df['date'].unique()

In [ ]:
# Cell: Import optimized marked Hawkes model
from MarkedBivariateHawkes_optimized import MarkedBivariateHawkesOptimized

# Cell: Process all dates with marked Hawkes (similar to all_day_results)
all_day_marked_results = {}

for i, target_date in enumerate(dates):
    print(f"Processing {i+1}/{len(dates)}: {target_date}...")
    
    # Prepare data for the day
    df_day = df[df['date'] == target_date].sort_values(['timestamp', 'trade_number'])
    market_open = pd.to_datetime((target_date) + ' 09:15:00')
    df_day['t'] = (df_day['timestamp'] - market_open).dt.total_seconds()
    
    # Extract buy and sell events with volumes
    buy_mask = df_day['aggressor_side'] == +1
    sell_mask = df_day['aggressor_side'] == -1
    
    buy_times = df_day[buy_mask]['t'].to_numpy()
    sell_times = df_day[sell_mask]['t'].to_numpy()
    buy_volumes = df_day[buy_mask]['volume'].to_numpy()
    sell_volumes = df_day[sell_mask]['volume'].to_numpy()
    
    # # Skip if insufficient data
    # if len(buy_times) < 10 or len(sell_times) < 10:
    #     print(f"  Skipping: insufficient events")
    #     continue
    
    try:
        # Fit marked Hawkes model
        model = MarkedBivariateHawkesOptimized()
        model.fit(buy_times, sell_times, buy_volumes, sell_volumes, 
                  optimize_beta=True, verbose=False)
        
        # Store results
        all_day_marked_results[target_date] = model.get_results()
        df_marked_results = pd.DataFrame(all_day_marked_results).T
        df_marked_results.to_csv('../data/all_day_marked_results_{}.csv'.format(target_date))
    except Exception as e:
        print(f"  Error: {e}")
        continue

print(f"\nCompleted: {len(all_day_marked_results)}/{len(dates)} dates")

# Cell: Save results as DataFrame
df_marked_results = pd.DataFrame(all_day_marked_results).T
df_marked_results.to_csv('../data/all_day_marked_results.csv')
print(f"Saved to: ../data/all_day_marked_results.csv")
print(df_marked_results.head())